<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/ML/Anomaly%20Detection/2_%D0%9E%D0%B1%D0%BD%D0%B0%D1%80%D1%83%D0%B6%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B0%D0%BD%D0%BE%D0%BC%D0%B0%D0%BB%D0%B8%D0%B9_%D1%81_%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E_%D0%BC%D0%B5%D1%82%D0%BE%D0%B4%D0%B0_Isolation_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Обнаружение аномалий с помощью метода Isolation Forest

## Введение

Обнаружение аномалий (Anomaly Detection) — это процесс идентификации наблюдений, которые существенно отличаются от остальных данных. Аномалии могут быть вызваны ошибками измерений, техническими сбоями, мошенничеством или другими необычными событиями. Основная задача обнаружения аномалий заключается в том, чтобы выделить такие "выбросы" из множества нормальных данных.

Метод **Isolation Forest** (iForest) — это алгоритм машинного обучения, который был разработан специально для обнаружения аномалий. Он основывается на концепции случайных деревьев и эффективно работает с большими объемами данных. В этой лекции мы подробно рассмотрим принцип работы этого метода, его преимущества и недостатки, а также практические примеры применения.



## 1. Постановка задачи обнаружения аномалий

### Что такое аномалия?
Аномалией называется объект, который значительно отличается от других объектов в датасете. Эти различия могут проявляться в различных характеристиках:
- **Отличие по значению**: Например, температура 50°C при средней температуре 20°C.
- **Отличие по поведению**: Например, резкий скачок трафика на сайте в определенное время.
- **Отличие по комбинации признаков**: Например, покупка дорогой вещи в неподходящее время с использованием чужой карты.

### Классификация методов обнаружения аномалий
Существует несколько подходов к решению задачи обнаружения аномалий:
1. **Статистические методы**: Используются распределения данных для определения выбросов.
2. **Методы на основе плотности**: Например, DBSCAN, LOF (Local Outlier Factor).
3. **Методы на основе близости**: Определяют аномалии через расстояние между точками.
4. **Методы на основе моделей**: Например, Autoencoders, PCA.
5. **Методы на основе деревьев**: Например, Isolation Forest.


## 2. Принцип работы Isolation Forest

### 2.1. Случайные деревья изоляции (Isolation Trees)

#### Основная идея
Каждое дерево изоляции строится путем **случайного разделения пространства признаков**. В отличие от традиционных деревьев решений, где разделение производится на основе оптимизации критериев (например, Gini или Information Gain), в дереве изоляции разделение выполняется **случайным образом**.

#### Процесс построения дерева:
1. **Выбор случайного признака**: На каждом уровне дерева выбирается один из доступных признаков $ X_i $ случайным образом.
2. **Выбор случайного порога**: Для выбранного признака $ X_i $ генерируется случайное значение $ p $ в пределах диапазона значений этого признака. Например, если значения признака находятся в интервале $[a, b]$, то $ p \in [a, b] $.
3. **Разделение данных**: Все объекты делятся на две группы:
   - Объекты, для которых $ X_i \leq p $, попадают в левую ветвь дерева.
   - Объекты, для которых $ X_i > p $, попадают в правую ветвь дерева.
4. **Повторение процесса**: Процедура повторяется рекурсивно для каждой группы до тех пор, пока:
   - Группа не содержит более одного объекта.
   - Или достигнута максимальная глубина дерева (параметр `max_depth`).

#### Пример построения дерева:
Допустим, у нас есть данные с двумя признаками $ X_1 $ и $ X_2 $, и их значения находятся в следующих диапазонах:
- $ X_1 \in [0, 10] $
- $ X_2 \in [5, 15] $

На первом шаге:
1. Выбирается случайный признак, например, $ X_1 $.
2. Генерируется случайное значение $ p = 7 $.
3. Данные делятся на две группы:
   - Левая ветвь: $ X_1 \leq 7 $
   - Правая ветвь: $ X_1 > 7 $

Процесс повторяется для каждой группы, пока все объекты не будут изолированы.



### 2.2. Почему аномалии изолируются быстрее?

Основная концепция метода состоит в том, что **аномалии находятся в областях с меньшей плотностью данных**, а нормальные объекты сконцентрированы в областях с высокой плотностью. Поскольку разделения выполняются случайным образом, аномальные объекты чаще оказываются в малочисленных группах и изолируются быстрее.

#### Как это работает?
1. **Нормальные объекты**: Они обычно находятся в центральных областях данных, где плотность высока. Чтобы изолировать такие объекты, требуется много разделений, так как они окружены другими объектами.
2. **Аномальные объекты**: Они находятся на периферии данных, где плотность низкая. Поэтому их можно отделить всего несколькими случайными разделениями.

#### Интуитивное объяснение:
Представьте себе карту города, где большинство людей живут в центре (нормальные объекты). Если вы случайно проводите границы через город, вам потребуется много попыток, чтобы отделить одного человека из центра. Однако люди, живущие на окраинах (аномалии), легко отделяются уже после нескольких случайных границ.



### 2.3. Алгоритм Isolation Forest

Isolation Forest использует ансамблевый подход: строится несколько деревьев изоляции, и результаты всех деревьев объединяются для получения финальной оценки аномальности каждого объекта.

#### Шаги алгоритма:

1. **Генерация случайных деревьев**:
   - Создается $ T $ деревьев изоляции (где $ T $ — параметр числа деревьев).
   - Каждое дерево строится независимо друг от друга, используя случайные выборки данных.

2. **Вычисление пути изоляции**:
   - Для каждого объекта $ x $ вычисляется средняя глубина изоляции $ H(x) $, на которой он был изолирован во всех деревьях.
   - Глубина изоляции $ h_t(x) $ для конкретного дерева $ t $ — это количество разделений, необходимых для изоляции объекта $ x $.

3. **Оценка аномальности**:
   - Чем меньше средняя глубина изоляции $ H(x) $, тем выше вероятность того, что объект является аномалией.
   - Финальная оценка аномальности может быть преобразована в вероятность или бинарный класс (аномалия/не аномалия).

#### Пример:
Пусть у нас есть три дерева, и для объекта $ x $ глубины изоляции равны:
- В первом дереве: $ h_1(x) = 5 $
- Во втором дереве: $ h_2(x) = 6 $
- В третьем дереве: $ h_3(x) = 4 $

Тогда средняя глубина изоляции:
$$
H(x) = \frac{h_1(x) + h_2(x) + h_3(x)}{T} = \frac{5 + 6 + 4}{3} = 5
$$

Если $ H(x) $ ниже средней глубины для всех объектов, то объект считается аномалией.



### 2.4. Формальная формулировка

#### Глубина изоляции
Пусть $ h(x) $ — это глубина узла, где объект $ x $ был изолирован в одном дереве. Тогда средняя глубина изоляции для объекта $ x $ в ансамбле из $ T $ деревьев вычисляется как:
$$
H(x) = \frac{1}{T} \sum_{t=1}^T h_t(x)
$$
где:
- $ h_t(x) $ — глубина изоляции объекта $ x $ в $ t $-м дереве.
- $ T $ — общее число деревьев.

#### Ожидаемая глубина изоляции
Для нормальных объектов ожидаемая глубина изоляции $ c(n) $ зависит от количества объектов $ n $ в выборке. Она может быть аппроксимирована формулой:
$$
c(n) = 2H(n-1) - (2(n-1)/n)
$$
где $ H(n-1) $ — гармоническое число:
$$
H(n-1) = \sum_{k=1}^{n-1} \frac{1}{k}
$$

#### Нормализованная оценка аномальности
Чтобы получить нормализованную оценку аномальности, используется следующая формула:
$$
S(x, n) = 2^{-\frac{H(x)}{c(n)}}
$$
где:
- $ S(x, n) $ — оценка аномальности объекта $ x $.
- $ H(x) $ — средняя глубина изоляции объекта.
- $ c(n) $ — ожидаемая глубина изоляции для нормальных объектов.

Значения $ S(x, n) $ лежат в диапазоне $[0, 1]$:
- $ S(x, n) \approx 1 $: объект является аномалией.
- $ S(x, n) \approx 0 $: объект является нормальным.



### 2.5. Особенности реализации

#### Параметры алгоритма:
1. **`n_estimators`**: Количество деревьев в ансамбле ($ T $).
   - Больше деревьев увеличивают точность, но замедляют работу.
2. **`max_samples`**: Размер выборки для каждого дерева.
   - По умолчанию используется $ 256 $ объектов.
3. **`contamination`**: Предполагаемая доля аномалий в данных.
   - Используется для установки порога классификации.
4. **`max_features`**: Количество признаков, используемых для разделения на каждом шаге.
   - По умолчанию используется все признаки.

#### Преимущества случайных деревьев:
- **Эффективность**: Разделение выполняется случайным образом, что делает процесс быстрым.
- **Масштабируемость**: Метод работает эффективно даже с большими объемами данных.



### 2.6. Графическая интерпретация

#### Пример двумерных данных:
Представьте себе набор точек на плоскости:
- Нормальные точки сосредоточены вокруг центра.
- Аномальные точки находятся на периферии.

Случайные деревья изоляции работают следующим образом:
1. Первое разделение делит пространство на две части.
2. Второе разделение делит каждую часть еще на две.
3. Этот процесс продолжается, пока все точки не будут изолированы.

Аномальные точки изолируются быстрее, так как они находятся в менее плотных областях.





## 3. Преимущества и недостатки Isolation Forest

### Преимущества:
1. **Эффективность**: Isolation Forest работает быстро даже с большими объемами данных благодаря логарифмической сложности изоляции объектов.
2. **Независимость от распределения данных**: Метод не предполагает никаких предположений о форме распределения данных.
3. **Простота реализации**: Алгоритм легко реализовать и использовать.
4. **Многомерность**: Подходит для многомерных данных без необходимости предварительной обработки.

### Недостатки:
1. **Чувствительность к параметрам**: Результат зависит от количества деревьев ($ T $) и максимальной глубины дерева.
2. **Неустойчивость к шуму**: В некоторых случаях шумовые данные могут быть ошибочно классифицированы как аномалии.
3. **Трудность интерпретации**: Хотя метод показывает, какие объекты являются аномалиями, он не объясняет, почему именно эти объекты выбиваются.



## 4. Практическая реализация

Давайте рассмотрим, как можно реализовать Isolation Forest на практике с использованием библиотеки `scikit-learn` в Python.

```python
from sklearn.ensemble import IsolationForest
import numpy as np
import matplotlib.pyplot as plt

# Генерация синтетических данных
np.random.seed(42)
X = 0.3 * np.random.randn(100, 2)  # Нормальные данные
X_outliers = np.random.uniform(low=-4, high=4, size=(20, 2))  # Аномалии
X = np.r_[X + 2, X - 2, X_outliers]  # Объединение нормальных данных и аномалий

# Обучение модели Isolation Forest
clf = IsolationForest(contamination=0.1, random_state=42)
clf.fit(X)

# Предсказание аномалий
y_pred = clf.predict(X)
anomalies = X[y_pred == -1]

# Визуализация результатов
plt.scatter(X[:, 0], X[:, 1], c='blue', s=20, label='Normal')
plt.scatter(anomalies[:, 0], anomalies[:, 1], c='red', s=50, marker='x', label='Anomalies')
plt.legend()
plt.show()
```

### Объяснение кода:
1. Мы генерируем синтетический датасет с нормальными данными и аномалиями.
2. Создаем модель Isolation Forest, указывая параметр `contamination`, который определяет долю аномалий в данных.
3. Обучаем модель и делаем предсказания.
4. Визуализируем результаты, где аномалии отмечены красным цветом.



## 5. Примеры применения

### 5.1. Обнаружение мошенничества
Isolation Forest может использоваться для обнаружения подозрительных транзакций в банковских системах. Например, если клиент внезапно совершает покупку на крупную сумму в непривычном месте, это может быть классифицировано как аномалия.

### 5.2. Мониторинг производственных процессов
В промышленности метод применяется для обнаружения дефектов в продукции или неисправностей оборудования. Например, если температура двигателя резко возрастает, это может сигнализировать о проблеме.

### 5.3. Анализ сетевых данных
Isolation Forest помогает выявлять атаки на сети, такие как DDoS-атаки, анализируя паттерны трафика.



Для лучшего понимания принципа работы метода Isolation Forest, давайте разберем конкретный числовой пример без использования кода. Мы будем использовать формулы и шаги алгоритма для вычисления аномальности объектов.



## Задача

У нас есть набор данных с двумя признаками $ X_1 $ и $ X_2 $. Всего 6 объектов:

| Объект | $ X_1 $ | $ X_2 $ |
|--------|----------|----------|
| $ x_1 $ | 1        | 2        |
| $ x_2 $ | 2        | 3        |
| $ x_3 $ | 3        | 4        |
| $ x_4 $ | 4        | 5        |
| $ x_5 $ | 5        | 6        |
| $ x_6 $ | 10       | 12       |

Здесь объект $ x_6 $ является потенциальной аномалией, так как он сильно отличается от остальных объектов.



## Шаг 1: Построение дерева изоляции

### Первое дерево
1. **Выбор случайного признака**: Выбираем $ X_1 $.
2. **Выбор случайного порога**: Генерируем порог $ p = 3.5 $.
3. **Разделение данных**:
   - Левая ветвь ($ X_1 \leq 3.5 $): $ x_1, x_2, x_3 $
   - Правая ветвь ($ X_1 > 3.5 $): $ x_4, x_5, x_6 $

#### Левая ветвь:
- Выбираем $ X_2 $.
- Генерируем порог $ p = 2.5 $.
- Разделяем данные:
  - Левая подветвь ($ X_2 \leq 2.5 $): $ x_1 $
  - Правая подветвь ($ X_2 > 2.5 $): $ x_2, x_3 $

- Для $ x_2, x_3 $ повторяем процесс:
  - Выбираем $ X_1 $.
  - Генерируем порог $ p = 2.5 $.
  - Разделяем данные:
    - Левая подветвь ($ X_1 \leq 2.5 $): $ x_2 $
    - Правая подветвь ($ X_1 > 2.5 $): $ x_3 $

#### Правая ветвь:
- Выбираем $ X_2 $.
- Генерируем порог $ p = 5.5 $.
- Разделяем данные:
  - Левая подветвь ($ X_2 \leq 5.5 $): $ x_4, x_5 $
  - Правая подветвь ($ X_2 > 5.5 $): $ x_6 $

- Для $ x_4, x_5 $ повторяем процесс:
  - Выбираем $ X_1 $.
  - Генерируем порог $ p = 4.5 $.
  - Разделяем данные:
    - Левая подветвь ($ X_1 \leq 4.5 $): $ x_4 $
    - Правая подветвь ($ X_1 > 4.5 $): $ x_5 $



## Шаг 2: Вычисление глубины изоляции

Глубина изоляции $ h(x) $ — это количество разделений, необходимых для полной изоляции объекта.

| Объект | Глубина изоляции ($ h(x) $) |
|--------|-----------------------------|
| $ x_1 $ | 2                           |
| $ x_2 $ | 3                           |
| $ x_3 $ | 3                           |
| $ x_4 $ | 3                           |
| $ x_5 $ | 3                           |
| $ x_6 $ | 2                           |

Обратите внимание, что $ x_6 $ был изолирован быстрее других объектов, что указывает на его потенциальную аномальность.



## Шаг 3: Средняя глубина изоляции

Пусть мы построили $ T = 3 $ дерева изоляции. Для каждого объекта вычислим среднюю глубину изоляции $ H(x) $.

Предположим, что результаты для всех трех деревьев следующие:

| Объект | $ h_1(x) $ | $ h_2(x) $ | $ h_3(x) $ | $ H(x) $ |
|--------|-------------|-------------|-------------|-----------|
| $ x_1 $ | 2          | 3          | 2          | 2.33      |
| $ x_2 $ | 3          | 3          | 3          | 3.00      |
| $ x_3 $ | 3          | 3          | 3          | 3.00      |
| $ x_4 $ | 3          | 3          | 3          | 3.00      |
| $ x_5 $ | 3          | 3          | 3          | 3.00      |
| $ x_6 $ | 2          | 2          | 2          | 2.00      |

Формула для средней глубины изоляции:
$$
H(x) = \frac{1}{T} \sum_{t=1}^T h_t(x)
$$



## Шаг 4: Ожидаемая глубина изоляции

Ожидаемая глубина изоляции $ c(n) $ для нормальных объектов рассчитывается по формуле:
$$
c(n) = 2H(n-1) - (2(n-1)/n)
$$
где $ H(n-1) $ — гармоническое число:
$$
H(n-1) = \sum_{k=1}^{n-1} \frac{1}{k}
$$

Для $ n = 6 $:
$$
H(5) = 1 + \frac{1}{2} + \frac{1}{3} + \frac{1}{4} + \frac{1}{5} \approx 2.283
$$
$$
c(6) = 2 \cdot 2.283 - \frac{2 \cdot 5}{6} \approx 4.566 - 1.667 \approx 2.899
$$



## Шаг 5: Нормализованная оценка аномальности

Нормализованная оценка аномальности $ S(x, n) $ вычисляется по формуле:
$$
S(x, n) = 2^{-\frac{H(x)}{c(n)}}
$$

Подставляем значения:

| Объект | $ H(x) $ | $ S(x, n) $ |
|--------|-----------|--------------|
| $ x_1 $ | 2.33      | $ 2^{-\frac{2.33}{2.899}} \approx 0.67 $ |
| $ x_2 $ | 3.00      | $ 2^{-\frac{3.00}{2.899}} \approx 0.50 $ |
| $ x_3 $ | 3.00      | $ 2^{-\frac{3.00}{2.899}} \approx 0.50 $ |
| $ x_4 $ | 3.00      | $ 2^{-\frac{3.00}{2.899}} \approx 0.50 $ |
| $ x_5 $ | 3.00      | $ 2^{-\frac{3.00}{2.899}} \approx 0.50 $ |
| $ x_6 $ | 2.00      | $ 2^{-\frac{2.00}{2.899}} \approx 0.75 $ |



## Заключение

На основе значений $ S(x, n) $ можно сделать вывод:
- Объекты $ x_2, x_3, x_4, x_5 $ имеют низкую оценку аномальности ($ S(x, n) \approx 0.50 $), что указывает на их нормальность.
- Объект $ x_6 $ имеет высокую оценку аномальности ($ S(x, n) \approx 0.75 $), что указывает на его аномальность.

Таким образом, метод Isolation Forest успешно обнаружил аномалию!




## Пример 1: Обнаружение аномалий в двумерных данных

### Задача
Мы создадим синтетический датасет с нормальными данными и добавим несколько аномалий. Затем применим Isolation Forest для их обнаружения.

### Код

```python
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

# Генерация синтетических данных
np.random.seed(42)
X = 0.3 * np.random.randn(100, 2)  # Нормальные данные (центральные точки)
X_outliers = np.random.uniform(low=-4, high=4, size=(20, 2))  # Аномалии (периферийные точки)
X = np.vstack((X + 2, X - 2, X_outliers))  # Объединение нормальных данных и аномалий

# Обучение модели Isolation Forest
clf = IsolationForest(contamination=0.1, random_state=42)  # Предполагаем, что 10% данных — аномалии
clf.fit(X)

# Предсказание аномалий
y_pred = clf.predict(X)  # Возвращает +1 для нормальных объектов и -1 для аномалий
anomalies = X[y_pred == -1]  # Выделяем аномалии
normal = X[y_pred == 1]  # Выделяем нормальные объекты

# Визуализация результатов
plt.figure(figsize=(8, 6))
plt.scatter(normal[:, 0], normal[:, 1], c='blue', s=20, label='Normal')
plt.scatter(anomalies[:, 0], anomalies[:, 1], c='red', s=50, marker='x', label='Anomalies')
plt.title('Isolation Forest Anomaly Detection')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.show()
```

### Объяснение кода:
1. **Генерация данных**:
   - Создаем нормальные данные как центральные точки, используя нормальное распределение.
   - Добавляем аномалии как периферийные точки, используя равномерное распределение.
2. **Обучение модели**:
   - Используем параметр `contamination=0.1`, чтобы указать, что предполагаем 10% аномалий в данных.
3. **Предсказание**:
   - Метод `predict` возвращает метки: `+1` для нормальных объектов и `-1` для аномалий.
4. **Визуализация**:
   - Нормальные объекты отображаются синими точками.
   - Аномалии отображаются красными крестами.



## Пример 2: Обнаружение аномалий в реальных данных

### Задача
Используем датасет `Boston Housing` из `scikit-learn`. Этот датасет содержит информацию о ценах на жилье в Бостоне. Мы будем искать аномалии среди признаков.

### Код

```python
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.ensemble import IsolationForest

# Загрузка датасета Boston Housing
boston = load_boston()
X = boston.data  # Признаки

# Обучение модели Isolation Forest
clf = IsolationForest(contamination=0.05, random_state=42)  # Предполагаем 5% аномалий
clf.fit(X)

# Предсказание аномалий
y_pred = clf.predict(X)  # Возвращает +1 для нормальных объектов и -1 для аномалий
anomalies_indices = np.where(y_pred == -1)[0]  # Индексы аномалий

# Визуализация аномалий по одному из признаков (например, CRIM)
plt.figure(figsize=(10, 6))
plt.scatter(range(len(X)), X[:, 0], c='blue', s=20, label='Normal')  # Признак CRIM
plt.scatter(anomalies_indices, X[anomalies_indices, 0], c='red', s=50, marker='x', label='Anomalies')
plt.title('Anomalies in Boston Housing Dataset (CRIM feature)')
plt.xlabel('Sample Index')
plt.ylabel('CRIM (Per capita crime rate)')
plt.legend()
plt.show()
```

### Объяснение кода:
1. **Загрузка данных**:
   - Используем датасет `Boston Housing`.
   - Выбираем все признаки для анализа.
2. **Обучение модели**:
   - Устанавливаем параметр `contamination=0.05`, так как предполагаем 5% аномалий.
3. **Предсказание**:
   - Находим индексы аномальных объектов.
4. **Визуализация**:
   - Отображаем значения одного из признаков (`CRIM`) для всех объектов.
   - Аномальные объекты выделяются красными крестами.


## Пример 3: Изменение параметров Isolation Forest

### Задача
Исследуем влияние параметра `contamination` на результаты детекции аномалий.

### Код

```python
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

# Генерация синтетических данных
np.random.seed(42)
X = 0.3 * np.random.randn(100, 2)  # Нормальные данные
X_outliers = np.random.uniform(low=-4, high=4, size=(20, 2))  # Аномалии
X = np.vstack((X + 2, X - 2, X_outliers))

# Функция для визуализации результатов
def plot_anomalies(X, contamination):
    clf = IsolationForest(contamination=contamination, random_state=42)
    clf.fit(X)
    y_pred = clf.predict(X)
    anomalies = X[y_pred == -1]
    normal = X[y_pred == 1]

    plt.figure(figsize=(8, 6))
    plt.scatter(normal[:, 0], normal[:, 1], c='blue', s=20, label='Normal')
    plt.scatter(anomalies[:, 0], anomalies[:, 1], c='red', s=50, marker='x', label='Anomalies')
    plt.title(f'Isolation Forest (contamination={contamination})')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.legend()
    plt.show()

# Тестирование разных значений contamination
plot_anomalies(X, contamination=0.05)  # 5% аномалий
plot_anomalies(X, contamination=0.1)   # 10% аномалий
plot_anomalies(X, contamination=0.2)   # 20% аномалий
```

### Объяснение кода:
1. **Функция визуализации**:
   - Для каждого значения параметра `contamination` строим модель Isolation Forest и визуализируем результаты.
2. **Тестирование**:
   - Проверяем, как меняется количество обнаруженных аномалий при различных значениях `contamination`.




 ##Isolation Forest для анализа временных рядов

В предыдущих частях мы рассмотрели основные принципы работы метода Isolation Forest, его применение для обнаружения аномалий в двумерных данных и реальных датасетах. Теперь перейдем к более сложной задаче — **обнаружению аномалий в временных рядах**. Временные ряды представляют собой последовательности значений, измеренных через равные промежутки времени, и часто используются в таких областях, как финансы, мониторинг систем, медицина и другие.



## 1. Особенности анализа временных рядов

Анализ временных рядов имеет свои особенности, которые необходимо учитывать при использовании методов обнаружения аномалий:

1. **Сезонность**: Временные ряды часто содержат повторяющиеся паттерны (например, суточные или недельные циклы).
2. **Тренд**: Данные могут иметь долгосрочную тенденцию к увеличению или уменьшению.
3. **Шум**: Реальные данные обычно содержат случайные флуктуации, которые не являются аномалиями.
4. **Контекст**: Аномалия в одном контексте может быть нормой в другом (например, высокая температура летом — это нормально, но зимой — аномалия).

Isolation Forest может эффективно работать с такими данными, если правильно подготовить их для анализа.


## 2. Подготовка данных для анализа временных рядов

Прежде чем применять Isolation Forest к временным рядам, необходимо выполнить несколько шагов подготовки данных:

### 2.1. Преобразование данных в формат признаков
Isolation Forest работает с многомерными данными, поэтому нужно преобразовать одномерный временной ряд в множество признаков. Это можно сделать несколькими способами:

#### a) Оконное преобразование (Sliding Window)
Оконное преобразование заключается в создании новых признаков на основе предыдущих значений ряда. Например, если у нас есть временной ряд $ [x_1, x_2, x_3, \dots] $, то можно создать окно размера 3:
$$
\text{Признаки: } [x_t, x_{t-1}, x_{t-2}]
$$
Это позволяет учесть зависимость между соседними точками.

#### b) Добавление дополнительных признаков
Можно добавить дополнительные признаки, такие как:
- Скользящее среднее.
- Стандартное отклонение за определенный период.
- Производная ряда.
- Индекс времени (например, час, день недели).

Пример:
```python
import numpy as np

# Исходный временной ряд
time_series = np.array([1, 2, 3, 4, 5, 6, 7, 8])

# Создание окна размера 3
window_size = 3
X = []
for i in range(window_size, len(time_series)):
    X.append(time_series[i - window_size:i])
X = np.array(X)

print("Преобразованные данные:\n", X)
```

Результат:
```
Преобразованные данные:
 [[1 2 3]
  [2 3 4]
  [3 4 5]
  [4 5 6]
  [5 6 7]]
```


### 2.2. Нормализация данных
Нормализация помогает улучшить работу алгоритма, особенно если значения временного ряда сильно различаются по масштабу. Обычно используется минимаксная нормализация:
$$
x' = \frac{x - \min(x)}{\max(x) - \min(x)}
$$
или стандартизация:
$$
x' = \frac{x - \mu}{\sigma}
$$
где $ \mu $ — среднее значение, $ \sigma $ — стандартное отклонение.



## 3. Применение Isolation Forest к временным рядам

После подготовки данных можно применить Isolation Forest для обнаружения аномалий.

### 3.1. Алгоритм
1. **Подготовка данных**: Преобразуйте временной ряд в формат признаков (например, используя оконное преобразование).
2. **Обучение модели**: Обучите модель Isolation Forest на подготовленных данных.
3. **Предсказание аномалий**: Используйте метод `predict` для классификации объектов как нормальных (`+1`) или аномальных (`-1`).
4. **Визуализация результатов**: Отобразите исходный временной ряд с выделенными аномалиями.


### 3.2. Пример реализации

#### Код
```python
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

# Генерация синтетического временного ряда
np.random.seed(42)
time = np.arange(0, 100, 0.5)  # Временная ось
data = np.sin(time) + np.random.normal(0, 0.1, len(time))  # Синусоида с шумом
outliers = np.random.randint(0, len(data), 10)  # Индексы выбросов
data[outliers] += np.random.uniform(2, 5, size=len(outliers))  # Добавляем выбросы

# Преобразование данных в формат для Isolation Forest
window_size = 3  # Размер окна
X = []
for i in range(window_size, len(data)):
    X.append(data[i - window_size:i])
X = np.array(X)

# Обучение модели Isolation Forest
clf = IsolationForest(contamination=0.05, random_state=42)  # Предполагаем 5% аномалий
clf.fit(X)

# Предсказание аномалий
y_pred = clf.predict(X)  # Возвращает +1 для нормальных объектов и -1 для аномалий
anomalies_indices = np.where(y_pred == -1)[0] + window_size  # Индексы аномалий

# Визуализация результатов
plt.figure(figsize=(10, 6))
plt.plot(time, data, c='blue', label='Time Series')
plt.scatter(time[anomalies_indices], data[anomalies_indices], c='red', s=50, marker='x', label='Anomalies')
plt.title('Isolation Forest for Time Series Anomaly Detection')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.show()
```

#### Объяснение кода:
1. **Генерация данных**:
   - Создаем синусоидальный временной ряд с небольшим шумом.
   - Добавляем случайные выбросы в определенные моменты времени.
2. **Преобразование данных**:
   - Используем оконное преобразование для создания признаков.
3. **Обучение модели**:
   - Устанавливаем параметр `contamination=0.05`, чтобы указать долю аномалий.
4. **Предсказание**:
   - Находим индексы аномалий.
5. **Визуализация**:
   - Отображаем временной ряд и выделяем аномалии красными крестами.



## 4. Анализ результатов

На графике видно, что Isolation Forest успешно обнаружил большинство добавленных аномалий. Однако важно отметить, что точность метода зависит от:
1. **Размера окна**: Большое окно может упустить короткие аномалии, а маленькое окно может увеличить количество ложных срабатываний.
2. **Параметра contamination**: Если установлен слишком низкий или слишком высокий уровень аномалий, это может повлиять на качество детекции.
3. **Качества данных**: Метод лучше работает с чистыми данными, где шум минимально влияет на результат.


## 5. Практические примеры применения

### 5.1. Финансовые данные
Isolation Forest может использоваться для обнаружения аномалий в ценовых графиках акций или валют. Например, резкие скачки цены могут быть вызваны ошибками торговли или мошенничеством.

### 5.2. Мониторинг производственных процессов
В промышленности метод применяется для обнаружения сбоев оборудования. Например, если температура двигателя внезапно возрастает, это может сигнализировать о проблеме.

### 5.3. Анализ энергопотребления
Isolation Forest помогает выявлять аномалии в потреблении электроэнергии, например, при необычно высоком или низком потреблении.
